In [1]:
#Semangat, fuck lah!!!!!!!!!!!!!!!!!!!!!!!!

In [4]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from textblob import TextBlob
 
ACCESS_TOKEN = "545740744-WV9TEBIcUD89VcD9eyYmYPIiM8GcLYvWwjGnSmoc"
ACCESS_TOKEN_SECRET = "7ZGS5pYNwtjDGDAEDc3nCO4NLhdFKf97u79yOTWx02S1G"
CONSUMER_KEY = "i1tyuoX3hURpMhc1TpX6JmuQB"
CONSUMER_SECRET = "YyFcD94eXxZNpULqqdHUkPaN09kSzrV0NxeXG2mgNGHeAUg4pf"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re


class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth


class TwitterStreamer():
    
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        stream.filter(track=hash_tag_list)


#kegabutan 

class TwitterListener(StreamListener):
    
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            
            return False
        print(status)


class TweetAnalyzer():
    

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="realDonaldTrump", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

                                                tweets                   id  \
0                            MAKE AMERICA GREAT AGAIN!  1163066735588720640   
1    With all that this Administration has accompli...  1163064539837022209   
2    .....”Journalism” has reached a new low in the...  1163063728218263552   
3    The Failing New York Times, in one of the most...  1163063723952676864   
4         RT @realDonaldTrump: https://t.co/JDS4zVfyBe  1162851091987488774   
5    RT @SharylAttkisson: I read that Trump rally w...  1162850561819103233   
6    RT @FullMeasureNews: Coming up this Sunday on ...  1162850237968506880   
7    RT @SharylAttkisson: From what I can tell, he ...  1162850126366412800   
8    RT @cleared37Joseph: SAVED 🇺🇸&amp;🌍Peace &amp;...  1162849601017323526   
9    RT @FullMeasureNews: Sunday on Full Measure: C...  1162849487884300289   
10   Such a disgrace at the once great @nytimes! ht...  1162848686797414400   
11   RT @twytter14: @tashlutsa @realDonaldTrump @CL.